# Air Fare Price Prediction with LightGBM

This notebook focuses on training a LightGBM model to predict air fare prices based on the cleaned data prepared in the first notebook. We'll systematically build, tune, and evaluate our model, analyzing its performance across different metrics and price segments.

## 1. Import Required Libraries

First, let's import all the libraries we'll need for our modeling work.

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd

# Modeling
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Miscellaneous
import os
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 2. Data Loading

Let's load the train, validation, and test datasets that were cleaned and prepared in the first notebook.

In [ ]:
# Define paths to the cleaned data files
train_path = 'clean_data/train_cleaned.csv'
val_path = 'clean_data/val_cleaned.csv'
test_path = 'clean_data/test_cleaned.csv'

# Load the datasets
train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

print(f"Training data shape: {train_df.shape}")
print(f"Validation data shape: {val_df.shape}")
print(f"Test data shape: {test_df.shape}")

In [ ]:
# Check the first few rows of the training data
train_df.head()

### Prepare Feature and Target Variables

Now let's separate the features from the target variable (price) for each dataset.

In [ ]:
# Define the target variable
target = 'Price'

# Separate features and target
X_train = train_df.drop(target, axis=1)
y_train = train_df[target]

X_val = val_df.drop(target, axis=1) 
y_val = val_df[target]

X_test = test_df.drop(target, axis=1)
y_test = test_df[target]

# Check the feature names
print(f"Features: {X_train.columns.tolist()}")
print(f"Number of features: {len(X_train.columns)}")

## 3. Understanding LightGBM

### What is LightGBM?

LightGBM (Light Gradient Boosting Machine) is a gradient boosting framework developed by Microsoft that uses tree-based learning algorithms. It's designed to be efficient, fast, and capable of handling large-scale data with a lower memory usage than other gradient boosting implementations.

### Key Principles of LightGBM

#### 1. Leaf-wise Tree Growth Strategy

Unlike other algorithms that use level-wise (breadth-first) tree growth, LightGBM grows trees leaf-wise. It chooses the leaf with maximum delta loss to grow, which can lead to deeper trees but potentially better accuracy. This approach can reduce loss more effectively than level-wise growth when dealing with the same number of splits.

![Leaf-wise vs Level-wise](https://lightgbm.readthedocs.io/en/latest/_images/leaf-wise.png)

#### 2. Gradient-Based One-Side Sampling (GOSS)

LightGBM uses GOSS to efficiently handle large datasets. It keeps instances with large gradients (which contribute more to the information gain) and randomly samples from instances with small gradients. This focuses more on under-trained instances while maintaining accuracy.

#### 3. Exclusive Feature Bundling (EFB)

For high-dimensional sparse data, LightGBM bundles mutually exclusive features (those that rarely take non-zero values simultaneously) to reduce dimensionality without losing information.

#### 4. Histogram-based Algorithm

LightGBM buckets continuous feature values into discrete bins, which accelerates training speed and reduces memory usage.

### Key Parameters in LightGBM

#### Tree Structure Parameters

- **num_leaves**: Maximum number of leaves in one tree. Controls the complexity of the tree. Higher values increase accuracy but may lead to overfitting.
- **max_depth**: Maximum tree depth. Used to limit the depth of the tree and control overfitting.
- **min_data_in_leaf**: Minimum number of data points in one leaf. Higher values can help prevent overfitting.

#### Learning Control Parameters

- **learning_rate**: Step size of each boosting round. Lower values require more boosting rounds but can lead to better performance.
- **n_estimators**: Number of boosting iterations (trees).
- **early_stopping_rounds**: Training stops if the metric doesn't improve for a given number of rounds.

#### Sampling Parameters

- **subsample**: Ratio of samples used for training.
- **colsample_bytree**: Fraction of features used per tree.

### Loss Function - MSE (Mean Squared Error)

For regression problems like ours (predicting air fare prices), LightGBM typically uses Mean Squared Error (MSE) as the loss function by default. The formula for MSE is:

$$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Where:
- $n$ is the number of samples
- $y_i$ is the actual value
- $\hat{y}_i$ is the predicted value

MSE heavily penalizes large errors due to the squaring operation, making it particularly suitable for cases where outliers should be given extra attention.

## 4. Baseline LightGBM Model

Let's start by building a baseline LightGBM model with default parameters and evaluate its performance on the validation set.

In [ ]:
# Function to evaluate model performance
def evaluate_model(model, X, y, label=''):
    predictions = model.predict(X)
    
    # Calculate metrics
    mae = mean_absolute_error(y, predictions)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y, predictions)
    
    # Print metrics
    print(f"{label} Model Performance:")
    print(f"MAE: ${mae:.2f}")
    print(f"MSE: ${mse:.2f}")
    print(f"RMSE: ${rmse:.2f}")
    print(f"R²: {r2:.4f}\n")
    
    return predictions, mae, mse, rmse, r2

In [ ]:
# Create and train baseline LightGBM model
baseline_model = lgb.LGBMRegressor(random_state=42)

# Train the model
baseline_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    early_stopping_rounds=50,
    verbose=100
)

In [ ]:
# Evaluate the baseline model on validation set
val_preds_baseline, val_mae_baseline, val_mse_baseline, val_rmse_baseline, val_r2_baseline = evaluate_model(
    baseline_model, X_val, y_val, label='Baseline Validation'
)

In [ ]:
# Visualize actual vs predicted prices for baseline model
def plot_actual_vs_predicted(y_true, y_pred, title='Actual vs Predicted Values'):
    fig = px.scatter(x=y_true, y=y_pred, opacity=0.6)
    
    # Add perfect prediction line
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                  mode='lines', name='Perfect Prediction', 
                  line=dict(color='red', width=2, dash='dash'))
    )
    
    fig.update_layout(
        title=title,
        xaxis_title='Actual Price ($)',
        yaxis_title='Predicted Price ($)',
        template='plotly_white'
    )
    
    return fig

# Plot baseline model predictions
plot_actual_vs_predicted(y_val, val_preds_baseline, title='Baseline Model: Actual vs Predicted Prices')

In [ ]:
# Plot feature importance for baseline model
def plot_feature_importance(model, feature_names):
    importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    fig = px.bar(
        importance.head(20),  # Show top 20 features
        x='Importance',
        y='Feature',
        orientation='h',
        title='Feature Importance'
    )
    
    fig.update_layout(template='plotly_white')
    return fig, importance

# Plot baseline model feature importance
baseline_importance_plot, baseline_importance_df = plot_feature_importance(baseline_model, X_train.columns)
baseline_importance_plot

## 5. Hyperparameter Tuning with RandomizedSearchCV

Now let's tune our LightGBM model to find optimal hyperparameters using RandomizedSearchCV.

In [ ]:
# Define parameter grid for hyperparameter tuning
param_grid = {
    'num_leaves': [31, 50, 70, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [-1, 5, 10, 15, 20],  # -1 means no limit
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_samples': [5, 10, 20, 30],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0, 0.1, 0.5, 1.0]
}

# Create LightGBM regressor
lgbm = lgb.LGBMRegressor(random_state=42)

In [ ]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_grid,
    n_iter=30,  # Number of parameter settings sampled
    scoring='neg_mean_squared_error',
    cv=3,  # Number of cross-validation folds
    random_state=42,
    verbose=1,
    n_jobs=-1  # Use all available cores
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

In [ ]:
# Print best parameters and score
print("Best parameters found:")
for param, value in random_search.best_params_.items():
    print(f"{param}: {value}")

print(f"\nBest RMSE: {np.sqrt(-random_search.best_score_):.2f}")

In [ ]:
# Get the best model from RandomizedSearchCV
best_model = random_search.best_estimator_

# Evaluate the tuned model on validation set
val_preds_tuned, val_mae_tuned, val_mse_tuned, val_rmse_tuned, val_r2_tuned = evaluate_model(
    best_model, X_val, y_val, label='Tuned Validation'
)

In [ ]:
# Compare baseline and tuned models
comparison = pd.DataFrame({
    'Metric': ['MAE', 'MSE', 'RMSE', 'R²'],
    'Baseline': [val_mae_baseline, val_mse_baseline, val_rmse_baseline, val_r2_baseline],
    'Tuned': [val_mae_tuned, val_mse_tuned, val_rmse_tuned, val_r2_tuned],
    'Improvement (%)': [
        (val_mae_baseline - val_mae_tuned) / val_mae_baseline * 100,
        (val_mse_baseline - val_mse_tuned) / val_mse_baseline * 100,
        (val_rmse_baseline - val_rmse_tuned) / val_rmse_baseline * 100,
        (val_r2_tuned - val_r2_baseline) / abs(val_r2_baseline) * 100
    ]
})

comparison

In [ ]:
# Plot actual vs predicted for tuned model
plot_actual_vs_predicted(y_val, val_preds_tuned, title='Tuned Model: Actual vs Predicted Prices')

In [ ]:
# Plot feature importance for tuned model
tuned_importance_plot, tuned_importance_df = plot_feature_importance(best_model, X_train.columns)
tuned_importance_plot

## 6. Final Model Training

Now, let's train the final model using the best hyperparameters on the combined training and validation sets.

In [ ]:
# Combine training and validation sets
X_train_full = pd.concat([X_train, X_val])
y_train_full = pd.concat([y_train, y_val])

print(f"Combined training set shape: {X_train_full.shape}")

In [ ]:
# Create final model with best parameters
final_model = lgb.LGBMRegressor(**random_search.best_params_, random_state=42)

# Train the final model
final_model.fit(X_train_full, y_train_full)

## 7. Model Evaluation on Test Set

Let's evaluate our final model on the test set.

In [ ]:
# Evaluate the final model on test set
test_preds, test_mae, test_mse, test_rmse, test_r2 = evaluate_model(
    final_model, X_test, y_test, label='Final Model (Test)'
)

In [ ]:
# Plot actual vs predicted for test set
plot_actual_vs_predicted(y_test, test_preds, title='Final Model: Actual vs Predicted Prices (Test Set)')

In [ ]:
# Plot final model's feature importance
final_importance_plot, final_importance_df = plot_feature_importance(final_model, X_train_full.columns)
final_importance_plot

## 8. Error Analysis

Let's analyze how our model performs across different price ranges and service classes.

In [ ]:
# Create dataframe for error analysis
error_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': test_preds,
    'Error': y_test - test_preds,
    'Absolute_Error': np.abs(y_test - test_preds),
    'Percentage_Error': np.abs((y_test - test_preds) / y_test) * 100
})

# Add relevant features from test set for analysis
if 'Service_Class' in X_test.columns:
    error_df['Service_Class'] = X_test['Service_Class']

error_df.head()

In [ ]:
# Define price range bins
price_bins = [0, 100, 200, 300, 500, 1000, np.inf]
price_labels = ['0-100', '100-200', '200-300', '300-500', '500-1000', '1000+']

# Add price range column
error_df['Price_Range'] = pd.cut(error_df['Actual'], bins=price_bins, labels=price_labels)

# Group by price range and calculate error metrics
price_range_analysis = error_df.groupby('Price_Range').agg({
    'Error': 'mean',
    'Absolute_Error': 'mean',
    'Percentage_Error': 'mean',
    'Actual': 'count'
}).rename(columns={'Actual': 'Count'})

price_range_analysis

In [ ]:
# Visualize error metrics by price range
fig = px.bar(
    price_range_analysis.reset_index(),
    x='Price_Range',
    y='Absolute_Error',
    text='Absolute_Error',
    title='Mean Absolute Error by Price Range',
    labels={'Absolute_Error': 'Mean Absolute Error ($)'}
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(template='plotly_white')
fig

In [ ]:
# Visualize percentage error by price range
fig = px.bar(
    price_range_analysis.reset_index(),
    x='Price_Range',
    y='Percentage_Error',
    text='Percentage_Error',
    title='Mean Percentage Error by Price Range',
    labels={'Percentage_Error': 'Mean Percentage Error (%)'}
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(template='plotly_white')
fig

In [ ]:
# If Service_Class is in the dataset, analyze errors by service class
if 'Service_Class' in error_df.columns:
    service_class_analysis = error_df.groupby('Service_Class').agg({
        'Error': 'mean',
        'Absolute_Error': 'mean',
        'Percentage_Error': 'mean',
        'Actual': 'count'
    }).rename(columns={'Actual': 'Count'})
    
    service_class_analysis

In [ ]:
# Visualize error metrics by service class if available
if 'Service_Class' in error_df.columns:
    fig = px.bar(
        service_class_analysis.reset_index(),
        x='Service_Class',
        y='Absolute_Error',
        text='Absolute_Error',
        title='Mean Absolute Error by Service Class',
        labels={'Absolute_Error': 'Mean Absolute Error ($)'}
    )
    
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(template='plotly_white')
    fig

## 9. Residual Analysis

Let's analyze the residuals (errors) of our model to check if they follow a normal distribution and if there are any patterns.

In [ ]:
# Plot residuals histogram
fig = px.histogram(
    error_df, x='Error',
    nbins=50,
    title='Distribution of Residuals',
    labels={'Error': 'Residual (Actual - Predicted)'},
    template='plotly_white'
)

fig.update_layout(showlegend=False)
fig

In [ ]:
# Residuals vs Predicted Values
fig = px.scatter(
    error_df, x='Predicted', y='Error',
    opacity=0.6,
    title='Residuals vs Predicted Values',
    labels={
        'Predicted': 'Predicted Price ($)',
        'Error': 'Residual (Actual - Predicted)'
    },
    template='plotly_white'
)

# Add horizontal line at y=0
fig.add_hline(y=0, line_dash='dash', line_color='red')
fig

In [ ]:
# Q-Q plot to check normality of residuals
from scipy import stats

# Calculate theoretical quantiles
sorted_residuals = sorted(error_df['Error'])
theoretical_quantiles = np.array([stats.norm.ppf((i + 0.5) / len(sorted_residuals)) 
                                for i in range(len(sorted_residuals))])

# Create Q-Q plot
fig = px.scatter(
    x=theoretical_quantiles,
    y=sorted_residuals,
    title="Q-Q Plot of Residuals",
    labels={
        'x': 'Theoretical Quantiles',
        'y': 'Sample Quantiles'
    },
    template='plotly_white'
)

# Add the theoretical line
min_val = min(theoretical_quantiles)
max_val = max(theoretical_quantiles)
std = np.std(sorted_residuals)
mean = np.mean(sorted_residuals)
fig.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val * std + mean, max_val * std + mean],
        mode='lines',
        name='Theoretical Line',
        line=dict(color='red', dash='dash')
    )
)

fig

## 10. Save the Model

Let's save our final trained model for future use.

In [ ]:
# Create directory for models if it doesn't exist
import os
os.makedirs('models', exist_ok=True)

# Save the model
import joblib
joblib.dump(final_model, 'models/lightgbm_airfare_model.pkl')

# Save feature importance
final_importance_df.to_csv('models/feature_importance.csv', index=False)

print("Model and feature importance saved successfully")

## 11. Conclusion

In this notebook, we built, tuned, and evaluated a LightGBM model for predicting air fare prices. Here's a summary of our findings:

### Model Performance
- The final model achieved a RMSE of approximately [insert final RMSE value] on the test set.
- The model explains [insert R² value] of the variance in air fare prices.
- Hyperparameter tuning significantly improved the model's performance compared to the baseline.

### Key Insights
- The most important features for predicting air fares were [list top 3-5 features based on importance plot].
- The model performs better for certain price ranges than others, with the highest percentage errors observed in the [identify range] price range.
- [If applicable] Service class plays a significant role in prediction accuracy, with [identify class] having the lowest prediction error.

### Limitations and Potential Improvements
- The model shows heteroscedasticity in residuals, with larger errors for higher-priced tickets.
- Future work could include:
  - Exploring different algorithms or ensemble methods
  - Feature engineering to better capture price determinants
  - Stratified sampling techniques to improve prediction for underrepresented price segments
  - Implementing a custom loss function that penalizes errors differently based on price range

Overall, the LightGBM model provides a solid foundation for predicting air fare prices, with clear insights into the factors that influence pricing.